# Notebook to Write Qualtrics Surveys

In [1]:
import pandas as pd
import numpy as np
import search_engine as ss #simple_search_func as ss
import re
import csv
import json

In [2]:
# Load up the qsf!
# with open('Query_Template.qsf', 'r') as file:
with open('Query_Template.qsf', 'r') as file:
    data = json.load(file)

In [3]:
choices_dict = {
    'love': ['hello', 'hi', 'are you', 'darling', 'sweetheart', 'honey', 'romance', 'affection', 'passion', 'devotion', 
             'adore', 'cherish', 'fondness', 'embrace', 'beloved', 'cuddle', 'valentine', 'kiss', 'heart', 'soulmate'],
    'sadness': ['meow', 'meow meow', 'meowwww', 'tears', 'gloom', 'despair', 'melancholy', 'sorrow', 'blue', 'grief', 
                'heartache', 'mourn', 'loneliness', 'misery', 'pain', 'regret', 'weep', 'hurt', 'lost', 'downcast'],
    'kiss': ['smoocher', 'kombucher', 'peck', 'smooch', 'liplock', 'mwah', 'pucker', 'osculation', 'romantic', 'passionate',
             'sweet', 'gentle', 'soft', 'warm', 'butterfly', 'cheek', 'forehead', 'eskimo', 'lovebite', 'chaste'],
    'happiness': ['joy', 'smile', 'laughter', 'cheer', 'delight', 'bliss', 'glee', 'ecstatic', 'merry', 'euphoria', 
                  'sunny', 'content', 'radiant', 'jubilant', 'bubbly', 'grateful', 'optimistic', 'carefree', 'vivacious', 'buoyant'],
    'anger': ['rage', 'fury', 'wrath', 'outrage', 'resentment', 'bitterness', 'spite', 'grudge', 'irritation', 'annoyance',
              'exasperation', 'hostility', 'hatred', 'indignation', 'fuming', 'seething', 'mad', 'infuriated', 'irate', 'tempest'],
    'fear': ['terror', 'panic', 'dread', 'fright', 'horror', 'alarm', 'anxiety', 'nervous', 'uneasy', 'apprehension',
             'timid', 'startle', 'tremble', 'phobia', 'shiver', 'quiver', 'spooked', 'petrified', 'chill', 'paranoia'],
    'surprise': ['shock', 'amazement', 'astonishment', 'wonder', 'startled', 'stunned', 'flabbergasted', 'aghast', 'dumbfounded', 'jolted',
                 'marvel', 'unexpected', 'perplexed', 'bewildered', 'gobsmacked', 'astounded', 'overwhelmed', 'speechless', 'baffled', 'staggered'],
    'friendship': ['bond', 'companion', 'ally', 'trust', 'loyal', 'camaraderie', 'mate', 'bro', 'sisterhood', 'brotherhood',
                   'affection', 'partnership', 'support', 'understanding', 'team', 'respect', 'connect', 'harmony', 'together', 'kindred'],
    'jealousy': ['envy', 'resent', 'covet', 'green-eyed', 'possessive', 'suspicion', 'mistrust', 'distrust', 'bitterness', 'spite',
                 'comparison', 'insecurity', 'longing', 'competitiveness', 'inferiority', 'grudge', 'fuming', 'anger', 'watchful', 'wary'],
    'nostalgia': ['memories', 'past', 'bittersweet', 'longing', 'reminisce', 'yearning', 'flashback', 'old days', 'sentimental', 'melancholy',
                  'childhood', 'throwback', 'wistful', 'retro', 'antique', 'classic', 'golden days', 'vintage', 'heartwarming', 'timeless'],
    'fear2': ['terror', 'panic', 'dread', 'fright', 'horror', 'alarm', 'anxiety', 'nervous', 'uneasy', 'apprehension',
             'timid', 'startle', 'tremble', 'phobia', 'shiver', 'quiver', 'spooked', 'petrified', 'chill', 'paranoia'],
    'surprise2': ['shock', 'amazement', 'astonishment', 'wonder', 'startled', 'stunned', 'flabbergasted', 'aghast', 'dumbfounded', 'jolted',
                 'marvel', 'unexpected', 'perplexed', 'bewildered', 'gobsmacked', 'astounded', 'overwhelmed', 'speechless', 'baffled', 'staggered'],
    'friendship2': ['bond', 'companion', 'ally', 'trust', 'loyal', 'camaraderie', 'mate', 'bro', 'sisterhood', 'brotherhood',
                   'affection', 'partnership', 'support', 'understanding', 'team', 'respect', 'connect', 'harmony', 'together', 'kindred'],
    'jealousy2': ['envy', 'resent', 'covet', 'green-eyed', 'possessive', 'suspicion', 'mistrust', 'distrust', 'bitterness', 'spite',
                 'comparison', 'insecurity', 'longing', 'competitiveness', 'inferiority', 'grudge', 'fuming', 'anger', 'watchful', 'wary'],
    'nostalgia2': ['memories', 'past', 'bittersweet', 'longing', 'reminisce', 'yearning', 'flashback', 'old days', 'sentimental', 'melancholy',
                  'childhood', 'throwback', 'wistful', 'retro', 'antique', 'classic', 'golden days', 'vintage', 'heartwarming', 'timeless']
}

In [4]:
topics = list(choices_dict.keys())# will be dict keys
counter = 0
next_topic=None


# iterate
for x in data['SurveyElements']:
    if type(x['Payload']) != dict:
        continue
    if 'QuestionType' not in x['Payload'].keys():
        continue
    if x['Payload']['QuestionType']=='MC': # group rank, influences others
        text = topics[counter]
        old_text = x['SecondaryAttribute']
        new_q = f'Select whether or not the following quotes are similar to the query "{text}"'
        x['Payload']['QuestionDescription']= new_q
        x['SecondaryAttribute'] = new_q
        x['Payload']['QuestionText']= new_q
        choices = choices_dict[text]
        new_choices = {str(x+1):{'Display': choices[x]} for x in range(len(choices))}
        x['Payload']['Choices']=new_choices
        x['Payload']['ChoiceOrder']=list(new_choices.keys())
        counter +=1
counter = 0
for x in data['SurveyElements']:
    if type(x['Payload']) != dict:
        continue
    if 'QuestionType' not in x['Payload'].keys():
        continue
    if x['Payload']['QuestionType']=='RO': # rank only, depends on other
        # THIS COMES FIRST
        # update Secondary Attribute: replace whatever is in ""
        text = topics[counter]
        next_topic = text
        old_text = x['SecondaryAttribute']
        new_q = f'Rank these quotes on how funny they are!'
        x['SecondaryAttribute'] = new_q
        # update ['Payload']['QuestionText'] replace whatever is in ""
        x['Payload']['QuestionText']= new_q
        x['Payload']['QuestionDescription']= new_q
        counter+=1
        #counter +=1

# save
with open('test.qsf', 'w') as json_file:
    json.dump(data, json_file) 

In [5]:
# make this a function

def new_survey_file(choices_dict:dict,file_name:str,template_name:str='Query_Template.qsf'):
    with open(template_name, 'r') as file:
        data = json.load(file)
    topics = list(choices_dict.keys())# will be dict keys
    c_topics = topics.copy()
    # surveyid = 'newstring'

    # iterate
    for x in data['SurveyElements']:
        if type(x['Payload']) != dict:
            continue
        if 'QuestionType' not in x['Payload'].keys():
            continue
        # x['SurveyID']= surveyid
        if x['Payload']['QuestionType']=='MC': # group rank, influences others
            text = topics.pop()
            new_q = f'Select whether or not the following quotes are similar to the query "{text}"'
            x['Payload']['QuestionDescription']= new_q
            x['SecondaryAttribute'] = new_q
            x['Payload']['QuestionText']= new_q
            choices = choices_dict[text]
            new_choices = {str(x+1):{'Display': choices[x]} for x in range(len(choices))}
            x['Payload']['Choices']=new_choices
            x['Payload']['ChoiceOrder']=list(new_choices.keys())
        if x['Payload']['QuestionType']=='RO': # rank only, depends on other
            # update Secondary Attribute: replace whatever is in ""
            text = c_topics.pop()
            new_q = f'Rank these quotes on how funny they are!'
            x['SecondaryAttribute'] = new_q
            # update ['Payload']['QuestionText'] replace whatever is in ""
            x['Payload']['QuestionText']= new_q
            x['Payload']['QuestionDescription']= new_q

    # save
    with open(file_name, 'w') as json_file:
        json.dump(data, json_file) 

### Load up examples

Now that I have the function written, I need to load up the search engine.

In [6]:
# load up that search
complete = pd.read_csv('https://scmcqueen.github.io/StarTrekScriptData/complete_data.csv')
complete.columns = ['index','character', 'quote', 'scene', 'location', 'view',
       'episode', 'date', 'series', 'file']
complete['quote']=complete['quote'].apply(lambda text: " ".join(text.split()))
complete['character']=complete['character'].fillna('NA')
# test_engine = ss.search_engine()
# test_engine.bulk_load(complete[['quote']].to_dict()['quote'])
# create instance of search engine
test_engine = ss.search_engine(name='BM25 Engine',full_data=complete)
# load data in bulk
test_engine.bulk_load(complete[['quote']].to_dict()['quote'])

We added 144211 documents. The engine now has 144211 documents.


In [7]:
results=test_engine.bw_search('candle',20,True)

In [8]:
keywords_1 =  ['candle','dream','doctor','bajor','prophets','inequality','struggle','engineer','prune','humanity']

In [10]:
indices = []
choices = []
search_term = []
bm25 = [] # new
bm25_prev = []
bm25_next = []
file_input = {}

for term in keywords_1:
    # do the search
    results=test_engine.bw_search(term,20,True)
    search_results = test_engine.old_pretty_print([x[0] for x in results])
    # update values
    t_ind = [x[0] for x in results]
    t_choices = [' '.join(x) for x in search_results]
    #t_choices = [test_engine.pretty_print(x[0]) for x in results]
    t_search = [term]*len(t_ind)
    # update big list
    indices+=t_ind
    choices+=t_choices
    search_term+=t_search
    bm25 +=[x[1] for x in results]
    bm25_prev +=[x[2] for x in results]
    bm25_next +=[x[3] for x in results]
    # update the dict
    file_input[term] = t_choices  

#new_survey_file(choices_dict=file_input,file_name='Evaluation/day_1.qsf',template_name='Query_Template.qsf')
pd.DataFrame({'indices':indices,'choices':choices,'search':search_term,'bm25':bm25,'prev_bm25':bm25_prev,'next_bm25':bm25_next}).to_csv('Evaluation/day_1_data.csv')

In [11]:
pd.DataFrame({'indices':indices,'choices':choices,'search':search_term,'bm25':bm25,'prev_bm25':bm25_prev,'next_bm25':bm25_next})

,indices,choices,search,bm25,prev_bm25,next_bm25
0,44542,JOSEPH: He stole a candle. O'BRIEN: One candle...,candle,14.147200,13.037027,0.0000
1,44541,O'BRIEN: What did he do to deserve this? JOSEP...,candle,13.037027,0.000000,14.1472
2,112905,TROI: Many things improve with age... maybe yo...,candle,11.491176,0.000000,0.0000
3,88378,"BEVERLY: Computer, secure door. BEVERLY: I lit...",candle,11.378741,0.000000,0.0000
4,88194,QUINT: There's a lot of things she didn't tell...,candle,11.054260,0.000000,0.0000
...,...,...,...,...,...,...
172,108760,"Q: After our last encounter, I was asked to le...",humanity,6.681643,0.000000,0.0000
173,79522,"""Q"" : You show promise, my good fellow. PICARD...",humanity,6.638868,0.000000,0.0000
174,119412,GEORDI: I'll be honest with you. We don't know...,humanity,6.354117,0.000000,0.0000
175,140566,TROI: Feelings aren't positive or negative Dat...,humanity,5.952888,0.000000,0.0000


In [ ]:
keywords_1 =  ['candle','dream','doctor','bajor','prophets','inequality','struggle','engineer','prune','humanity',
               'story', 'religion', 'child', 'molly','counselor']
keywords_2 =  ['archaeology','chateau','trial','fiction','romance','wedding','father','kahless','chocolate','future',
               'bridge','saucer','coffee','romulans','colony']
keywords_3 =  ['raktajino','commit no errors','tanagra','borg','cardassia','poetry','spot','assimilate','unusual','barclay',
               'acquisition','rules','vedek','warp crystal',"bat'leth"]
keywords_4 =  ['blood wine','gagh','cook','celebrate','impossible','earl grey','jake','geordi','riker','poker',
               'phaser','occupation','conscious minds','tailor','neutral zone']
keywords_5 =  ['men','women','miles','worf','bashir','commander','captain','kira','troi','guinan',
               'shields','unification','mister spock','tribble','android']
keywords_6 =  ['unraveled','friendship','hailing frequencies','universe','traveler','wesley','good tea','good book','make it so','flute',
               'neutrino levels','noonian soong','breen','evidence','vacation']
keywords_7 =  ['memory','merry man','obedient','honor','san francisco','new orleans','liberation','imagination','maquis','tribunal',
               'trill','lore','risa','detective','ezri']
keywords_8 =  ['not picard','ice cream','morn','weyoun','dukat','civilization','replicators','truth','lunch','past',
               'empath','betray','warp bubble','jadzia','pregnant']
keywords_9 =  ['vineyard','lwaxana','bucket','justice is justice','espionage','same lie','mother','klingon','baseball','moriarty',
               'vulcan','moon','lower decks','the academy','full impulse']
keywords_10 =  ['enterprise','ferengi','nagus','death','root beer','alpha quadrant','wormhole','tasha','holosuite','holmes',
                'the defiant','ensign ro','dabo girl','secret agent','oo-mox']

In [13]:
keywords = {1:keywords_1,
            2:keywords_2,
            3:keywords_3,
            4:keywords_4,
            5:keywords_5,
            6:keywords_6,
            7:keywords_7,
            8:keywords_8,
            9:keywords_9,
            10:keywords_10
            }

for x in keywords.keys():
    counter = str(x)
    t_keys = keywords[x]

    indices = []
    choices = []
    search_term = []
    bm25 = [] # new
    bm25_prev = []
    bm25_next = []
    file_input = {}

    for term in t_keys:
        # do the search
        results=test_engine.bw_search(term,20,True)
        #search_results = test_engine.pretty_print([x[0] for x in results])
        # update values
        t_ind = [x[0] for x in results]
        #t_choices = [' '.join(x) for x in search_results]
        t_choices = [test_engine.pretty_print(x[0]) for x in results]
        t_search = [term]*len(t_ind)
        # update big list
        indices+=t_ind
        choices+=t_choices
        search_term+=t_search
        bm25 +=[x[1] for x in results]
        bm25_prev +=[x[2] for x in results]
        bm25_next +=[x[3] for x in results]
        # update the dict
        file_input[term] = t_choices

    #new_survey_file(choices_dict=file_input,file_name=f'Evaluation/day_{counter}.qsf',template_name='Query_Template.qsf')
    pd.DataFrame({'indices':indices,'choices':choices,'search':search_term,'bm25':bm25,'prev_bm25':bm25_prev,'next_bm25':bm25_next}).to_csv(f'Evaluation/day_{counter}_data.csv')

# Upload Results


In [14]:
def get_results_df(results_path:str='Evaluation/Results/day2_skyeler_only.tsv',questions_path:str='Evaluation/day_2_data.csv'):
    # start by reading a file

    results = pd.read_csv(results_path)
    questions_ids = pd.read_csv(questions_path,index_col=0)
    questions_ids.columns = ['indices', 'quote', 'query','bm25','prevbm25','nextbm25']
    results =results.drop(columns=['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress', 'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId', 'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 'LocationLatitude', 'LocationLongitude', 'DistributionChannel', 'UserLanguage'])
    cols = results.columns

    ranking_cols = [x for x in list(cols) if '_' in x]
    q_col = [x for x in list(cols) if 'Q' in x and '_' not in x and 'Q1'!=x]
    questions = results[ranking_cols].T

    # # query+map should be diff
    questions = questions.reset_index()
    questions['query']=questions[0].apply(lambda x:re.findall(r"'(.*?)'", x)[0])
    questions['index']=questions['index'].apply(lambda x: x[:x.index('_')])
    query_map = questions[['index','query']]

    col_results = results[ranking_cols].T

    col_results.columns = ['question','id','ranking']
    col_results = col_results.reset_index()
    col_results['quote']=col_results['question'].apply(lambda x: x[x.index('-')+2:])
    col_results['index'] = col_results['index'].apply(lambda x: x[:x.index('_')])
    col_results['query']=col_results['question'].apply(lambda x:re.findall(r"'(.*?)'", x)[0])
    result = questions_ids.merge(col_results[['query','quote','ranking']],on=['query','quote'],how='left')
    result['ranking']=result['ranking'].fillna(-1)
    return result

In [ ]:
results_path = 'Evaluation/Results/day1_skyeler_only.csv'
questions_path='Evaluation/day_1_data.csv'

results = pd.read_csv(results_path)
questions_ids = pd.read_csv(questions_path,index_col=0)
questions_ids.columns = ['indices', 'quote', 'query','bm25','prevbm25','nextbm25']
results =results.drop(columns=['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress', 'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId', 'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 'LocationLatitude', 'LocationLongitude', 'DistributionChannel', 'UserLanguage'])
cols = results.columns

ranking_cols = [x for x in list(cols) if '_' in x]
q_col = [x for x in list(cols) if 'Q' in x and '_' not in x and 'Q1'!=x]
questions = results[ranking_cols].T

# # # query+map should be diff
questions = questions.reset_index()
questions['query']=questions[0].apply(lambda x:re.findall(r"'(.*?)'", x)[0])
#questions['index']=questions['index'].apply(lambda x: x[:x.index('_')])
query_map = questions[['index','query']]

col_results = results[ranking_cols].T

col_results.columns = ['question','id','ranking']
col_results = col_results.reset_index()
col_results['quote']=col_results['question'].apply(lambda x: x[x.index('-')+2:])
# col_results['index'] = col_results['index'].apply(lambda x: x[:x.index('_')])
# col_results['query']=col_results['question'].apply(lambda x:re.findall(r"'(.*?)'", x)[0])
# result = questions_ids.merge(col_results[['query','quote','ranking']],on=['query','quote'],how='left')
# result['ranking'].isna().sum()
# result['ranking']=result['ranking'].fillna(-1)

,index,0,1,2,query
0,Q3_1,Rank these quotes on how funny they are for th...,"{""ImportId"":""QID4_1""}",NaN,doctor
1,Q3_2,Rank these quotes on how funny they are for th...,"{""ImportId"":""QID4_2""}",NaN,doctor
2,Q3_3,Rank these quotes on how funny they are for th...,"{""ImportId"":""QID4_3""}",NaN,doctor
3,Q3_4,Rank these quotes on how funny they are for th...,"{""ImportId"":""QID4_4""}",NaN,doctor
4,Q3_5,Rank these quotes on how funny they are for th...,"{""ImportId"":""QID4_5""}",NaN,doctor
...,...,...,...,...,...
367,Q31_26,Rank these quotes on how funny they are for th...,"{""ImportId"":""QID32_26""}",NaN,bajor
368,Q31_27,Rank these quotes on how funny they are for th...,"{""ImportId"":""QID32_27""}",NaN,bajor
369,Q31_28,Rank these quotes on how funny they are for th...,"{""ImportId"":""QID32_28""}",NaN,bajor
370,Q31_29,Rank these quotes on how funny they are for th...,"{""ImportId"":""QID32_29""}",NaN,bajor


In [15]:
week1 =get_results_df(results_path='Evaluation/Results/day1_skyeler_only.csv',questions_path='Evaluation/day_1_data.csv')
week2 =get_results_df(results_path='Evaluation/Results/day2_skyeler_only.csv',questions_path='Evaluation/day_2_data.csv')
week3 =get_results_df(results_path='Evaluation/Results/day3_skyeler_only.csv',questions_path='Evaluation/day_3_data.csv')
week4 =get_results_df(results_path='Evaluation/Results/day4_skyeler_only.csv',questions_path='Evaluation/day_4_data.csv')
week5 =get_results_df(results_path='Evaluation/Results/day5_skyeler_only.csv',questions_path='Evaluation/day_5_data.csv')
week6 =get_results_df(results_path='Evaluation/Results/day6_skyeler_only.csv',questions_path='Evaluation/day_6_data.csv')
week7 =get_results_df(results_path='Evaluation/Results/day7_skyeler_only.csv',questions_path='Evaluation/day_7_data.csv')
week8 =get_results_df(results_path='Evaluation/Results/day8_skyeler_only.csv',questions_path='Evaluation/day_8_data.csv')
week9 =get_results_df(results_path='Evaluation/Results/day9_skyeler_only.csv',questions_path='Evaluation/day_9_data.csv')

/var/folders/r0/2fgtj8y934zddv46dgfbq1xw0000gn/T/ipykernel_2018/3448903697.py:28: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result['ranking']=result['ranking'].fillna(-1)
/var/folders/r0/2fgtj8y934zddv46dgfbq1xw0000gn/T/ipykernel_2018/3448903697.py:28: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result['ranking']=result['ranking'].fillna(-1)
/var/folders/r0/2fgtj8y934zddv46dgfbq1xw0000gn/T/ipykernel_2018/3448903697.py:28: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Cal

In [16]:
combined_eval_data = pd.concat([week9,week8,week7,week6, week5,week4,week3,week2,week1])
# combined_eval_data.to_csv('skyeler_ranking_data.csv')
combined_eval_data

,indices,quote,query,bm25,prevbm25,nextbm25,ranking
0,113689,TIMICIN: No. We've... we've said our good-byes...,lwaxana,12.436612,0.000000,0.00000,-1
1,113558,LWAXANA: I agree. How about letting everybody ...,lwaxana,12.436612,0.000000,0.00000,-1
2,113532,LWAXANA: I'm not sure. A minute. An hour. \n T...,lwaxana,12.436612,0.000000,0.00000,-1
3,55180,"ODO: Marry me, Lwaxana... let me into your lig...",lwaxana,12.436612,8.857818,0.00000,-1
4,55060,ODO: I'm sorry if I made you feel unwelcome......,lwaxana,12.436612,0.000000,0.00000,-1
...,...,...,...,...,...,...,...
257,112533,TROI: Sometimes... even when a victim has deal...,counselor,8.975723,0.000000,4.63639,-1
258,109313,WORF: But what do the Pakleds want? \n RIKER: ...,counselor,8.975723,0.000000,0.00000,-1
259,108851,BORG: We have analyzed your defensive capabili...,counselor,8.975723,0.000000,0.00000,-1
260,105620,TROI: Where's Lieutenant Monroe? \n O'BRIEN: C...,counselor,8.975723,0.000000,0.00000,-1


# COME BACK & FIX THIS

In [24]:
temp = pd.read_csv('skyeler_ranking_data_correct.csv',index_col=0)[['indices','ranking','query']]
combined_eval_data.drop(columns=['ranking']).merge(temp,on=['indices','query'],how='left').to_csv('skyeler_ranking_data.csv')

In [18]:
combined_eval_data['ranking']=combined_eval_data['ranking'].apply(lambda x: int(x))
combined_eval_data.ranking.describe()

count    2461.0
mean       -1.0
std         0.0
min        -1.0
25%        -1.0
50%        -1.0
75%        -1.0
max        -1.0
Name: ranking, dtype: float64